In [ ]:
#XGB BOOST - best algortithm #better than gradient #good for complx probs
#used for regression
#1. The main advantages of XGBoost is its lightning speed compared to other algorithms
#2. It uses parallel processing to increase the speed.
#3. Sequential tree growing
#4. However, XGBoost is more difficult to understand, visualize and to tune compared to AdaBoost and random forests.
#5. XGBoost also handles missing values in the dataset. So, in data wrangling you may or may not do
#a separate treatment for the missing values because XGBoost is capable of handling missing values internally.
#6. XGBoost is very popular because it has been the winning algorithm in a number of Kaggle competitions

In [3]:
!pip install xgboost

     ------------------------------------ 125.4/125.4 MB 836.9 kB/s eta 0:00:00


In [24]:
#Used Cars Price Prediction
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
# metrics and models
from sklearn.metrics import r2_score, mean_squared_error
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore")

In [25]:
df = pd.read_csv('vehicles.csv')
df.head(5)

,Unnamed: 0,id,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,county,state,lat,long
0,55483,7315914053,0,2018.0,ram,promaster 2500,excellent,NaN,gas,44244.0,clean,automatic,NaN,NaN,van,NaN,NaN,ca,32.792800,-116.966500
1,162368,7310885048,13995,2017.0,mazda,cx-3,NaN,4 cylinders,gas,7037.0,rebuilt,automatic,NaN,NaN,SUV,white,NaN,ia,41.207382,-96.023096
2,234393,7308243856,19990,2019.0,mitsubishi,eclipse cross sp,good,NaN,gas,35313.0,clean,other,4wd,NaN,hatchback,white,NaN,nc,35.190000,-80.830000
3,276110,7315817729,0,2019.0,honda,cr-v,NaN,NaN,gas,25626.0,clean,automatic,NaN,NaN,SUV,orange,NaN,ny,40.854573,-74.120219
4,349033,7301620999,42900,2015.0,chevrolet,corvette,excellent,8 cylinders,gas,29000.0,clean,automatic,NaN,NaN,convertible,black,NaN,sc,34.755562,-82.906419


In [26]:
df.tail(10)

,Unnamed: 0,id,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,county,state,lat,long
64022,97133,7311606327,5500,2008.0,nissan,altima 2.5,good,4 cylinders,gas,168000.0,clean,automatic,NaN,NaN,NaN,NaN,NaN,fl,29.772800,-82.378200
64023,179669,7310752246,23990,2014.0,gmc,sierra,NaN,NaN,other,122969.0,clean,automatic,4wd,NaN,pickup,black,NaN,me,42.834977,-71.107791
64024,389527,7311009263,27990,2017.0,buick,enclave leather sport,good,6 cylinders,gas,31479.0,clean,other,fwd,NaN,other,black,NaN,va,38.030000,-78.480000
64025,53491,7315718962,16590,2016.0,ford,c-max energi sel wagon 4d,good,NaN,other,53180.0,clean,other,NaN,NaN,wagon,black,NaN,ca,38.580000,-121.490000
64026,368961,7316776290,13991,2017.0,toyota,corolla,NaN,4 cylinders,gas,79964.0,clean,automatic,fwd,mid-size,sedan,NaN,NaN,tx,33.034224,-96.811608
64027,250960,7310954770,3495,1997.0,mercury,grand marquis,good,8 cylinders,gas,106253.0,clean,automatic,rwd,NaN,sedan,white,NaN,nj,40.588352,-74.291039
64028,217058,7305666517,25590,2018.0,lincoln,mkz reserve hybrid sedan,good,NaN,hybrid,33467.0,clean,other,fwd,NaN,sedan,white,NaN,mn,44.010000,-92.470000
64029,323468,7316544829,22421,2015.0,mercedes-benz,e-class,excellent,NaN,gas,55494.0,clean,automatic,rwd,NaN,sedan,black,NaN,or,45.504350,-122.532962
64030,132003,7314796788,29999,2013.0,jeep,wrangler unlimited sport,NaN,NaN,gas,63408.0,clean,automatic,4wd,NaN,SUV,white,NaN,id,43.619740,-116.294690
64031,296896,7309354482,8850,2015.0,chevrolet,sonic,excellent,4 cylinders,gas,114248.0,clean,automatic,fwd,sub-compact,sedan,green,NaN,oh,40.002779,-83.133361


In [27]:
#duplicte entry is also possible as peplw repost their ads
drop_columns = ['Unnamed: 0', 'id', 'title_status', 'size', 'county', 'state', 'lat', 'long']
df = df.drop(columns = drop_columns, axis=1)

In [28]:
df.shape


(64032, 12)

In [29]:
df.isna().sum()

price               0
year              158
manufacturer     2569
model             802
condition       26097
cylinders       26511
fuel              424
odometer          669
transmission      353
drive           19471
type            13785
paint_color     19505
dtype: int64

In [31]:
#as the data is very big, we cn get rid of nan. Also adding null to so many data is painful
df=df.dropna()
df.head()

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color
5,0,2006.0,chrysler,300,like new,8 cylinders,gas,149000.0,automatic,rwd,sedan,white
9,20995,2011.0,chevrolet,silverado 1500,excellent,8 cylinders,gas,92001.0,automatic,4wd,truck,blue
15,50995,2017.0,gmc,yukon denali,like new,8 cylinders,gas,70227.0,automatic,4wd,SUV,grey
22,13500,2014.0,chevrolet,tahoe,good,8 cylinders,gas,96007.0,automatic,rwd,SUV,white
29,34990,2016.0,gmc,canyon crew cab sle pickup,good,6 cylinders,gas,34425.0,other,4wd,pickup,red


In [32]:
df.describe()
#this model will capture only continous variables

,price,year,odometer
count,1.749100e+04,17491.000000,1.749100e+04
mean,1.632545e+04,2009.535247,1.122616e+05
std,1.294429e+05,9.596493,2.155149e+05
min,0.000000e+00,1918.000000,0.000000e+00
25%,5.600000e+03,2006.000000,5.578700e+04
50%,1.095000e+04,2012.000000,1.025670e+05
75%,2.250000e+04,2015.000000,1.480000e+05
max,1.700000e+07,2022.000000,1.000000e+07


In [33]:
#removing duplicates, since there will be a lot of them
df.drop_duplicates(inplace=True)

In [34]:
df.shape

(15035, 12)

In [35]:
#lets focus on categorical data
numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
categorical_columns = []
features = df.columns.values.tolist()
for col in features:
    if df[col].dtype in numerics:
        continue
    categorical_columns.append(col)

In [36]:
categorical_columns


['manufacturer',
 'model',
 'condition',
 'cylinders',
 'fuel',
 'transmission',
 'drive',
 'type',
 'paint_color']

In [37]:
df_dummies = pd.get_dummies(df[categorical_columns], drop_first=True)

In [39]:
df_dummies.head()

,manufacturer_alfa-romeo,manufacturer_audi,manufacturer_bmw,manufacturer_buick,manufacturer_cadillac,manufacturer_chevrolet,manufacturer_chrysler,manufacturer_datsun,manufacturer_dodge,manufacturer_ferrari,...,paint_color_brown,paint_color_custom,paint_color_green,paint_color_grey,paint_color_orange,paint_color_purple,paint_color_red,paint_color_silver,paint_color_white,paint_color_yellow
5,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
22,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [40]:
df_dummies.shape

(15035, 4267)

In [41]:
df=df.join(df_dummies)

In [42]:
df.shape

(15035, 4279)

In [45]:
df.drop(columns=categorical_columns,axis=1, inplace=True)

In [46]:
df.head (2)

,price,year,odometer,manufacturer_alfa-romeo,manufacturer_audi,manufacturer_bmw,manufacturer_buick,manufacturer_cadillac,manufacturer_chevrolet,manufacturer_chrysler,...,paint_color_brown,paint_color_custom,paint_color_green,paint_color_grey,paint_color_orange,paint_color_purple,paint_color_red,paint_color_silver,paint_color_white,paint_color_yellow
5,0,2006.0,149000.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
9,20995,2011.0,92001.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
df = df[df['price'] > 1000] #set a min price to have realistic sensible data
df = df[df['price'] < 40000] #set a max price to have realistic sensible data
df.shape
#While practicing try other encoding techniques as well and compare the final accuracy.
#Selecting realistic data. Here Domain knowledge will help a lot to decide what could be the higher and lower
#price.

(13548, 4270)

In [48]:
#divide it into features and label
y = df['price']
X = df.drop(['price'], axis=1)

In [49]:
# Data split into train test
train_X, test_x, train_Y, test_y= train_test_split(X, y, test_size=0.25, random_state=10)

In [50]:
#xgb
import xgboost as xgb
xgb=xgb.XGBRegressor()

In [51]:
xgb.fit(train_X, train_Y) #tkes time 10-20 min

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [53]:
y_pred=xgb.predict(test_x)

In [54]:
r2_score(test_y,y_pred)
#Tune the parameters and check if you can increase the score

0.8072729917647082

In [ ]:
Bagging                                                                       Boosting
Bagging takes indivdual trress or decision based on idividual groups    Boosting takes decision beased o the previous step, first step starts form scrathch


Training data subsets are drawn randomly with                 Each new subset contains the components
replacement from the entire training dataset.              that were misclassified by previous models.
Bagging attempts to tackle the over-fitting                Boosting tries to reduce bias.
issue.
Every model receives an equal weight.                      Models are weighted by their performance.
Objective to decrease variance, not bias.               Objective to decrease bias, not variance.
Every model is built independently.                   New models are affected by the performance of the previously developed model